In [46]:
import json
import pandas as pd

match_id = "EFBO2372905"

with open("./matches/EFBO2372905.json") as f:
    data = f.read()

data = json.loads(data)
match_key = next(iter(data["matchData"][0]["tournamentDatesWithEvents"]))
game_date = data["matchData"][0]["tournamentDatesWithEvents"][match_key][0]["events"][0]["startTime"][:10]


with open(f"./line-ups/{match_id}.json") as f:
    data = f.read()

data = json.loads(data)
home_team = data[0]["data"]["teams"]["homeTeam"]["name"]
away_team = data[0]["data"]["teams"]["awayTeam"]["name"]

if home_team == "Tranmere Rovers":
    team = 'homeTeam'
else:
    team = 'awayTeam'

formation = data[0]["data"]["teams"][team]["formation"]
formation = "-".join(str(formation))

players = data[0]["data"]["teams"][team]["players"]

apps_df = pd.DataFrame()
cards_df = pd.DataFrame()
for player in players:
    player_name = player["name"]["full"]
    shirt_no = player["meta"]["uniformNumber"]
    role = player["meta"]["status"].replace("bench", "sub")
    player_app = {
        "game_date": game_date,
        "player_name": player_name,
        "shirt_no": player_name,
        "role": role
    }
    player_app = pd.DataFrame(player_app, index = [0])

    if player["meta"]["status"] == "bench" and not player["substitutions"]:
        print(f"{player['name']['full']} did not come on")
    
    # role = player["meta"]["status"]
    # shirt_no = player["meta"]["uniformNumber"]
    # player_pos = player["meta"]["positionRegular"]
    apps_df = pd.concat([apps_df, player_app], axis = 0)

    cards = player["bookings"]
    if cards:
        player_cards = pd.json_normalize(player["bookings"])
        player_cards["player_name"] = player_name
        cards_df = pd.concat([cards_df, player_cards], axis = 0)

cards_df
apps_df

Luke McGee did not come on
Lee O'Connor did not come on
Jordan Turnbull did not come on
Kane Hemmings did not come on
Josh Hawkes did not come on
Charlie Jolley did not come on
Chris Merrie did not come on
Connor Jennings did not come on
Harvey Saunders did not come on


,game_date,player_name,shirt_no,role
0,2023-08-08,Joe Murphy,13,starter
0,2023-08-08,Tom Davies,5,starter
0,2023-08-08,Josef Yarney,21,starter
0,2023-08-08,Jake Leake,3,starter
0,2023-08-08,Reece McAlear,23,starter
0,2023-08-08,Kieron Morris,7,starter
0,2023-08-08,Paul Lewis,22,starter
0,2023-08-08,Regan Hendry,8,starter
0,2023-08-08,Samuel Taylor,20,starter
0,2023-08-08,Kristian Dennis,14,starter


In [18]:
pd.json_normalize(player["bookings"])

,type,timeElapsed,addedTime
0,yellow-card,69,0


In [44]:
pd.DataFrame(player_app)

,game_date,player_name,shirt_no,role
0,2023-08-08,Harvey Saunders,19,sub
